<a href="https://colab.research.google.com/github/salonip/ChatBots/blob/master/chatbot_with_CNN_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import json
import numpy as np 
import nltk 
import tensorflow as tf
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


This approach understands the spelling mistakes better and has a better contextual understanding

In [3]:
import os
project_path = "/content/drive/My Drive/chatbot/"
os.chdir(project_path)

In [4]:

with open("intents.json") as file:
    bot = json.load(file)

In [5]:
labels = []
intent_part_x = []
intent_part_y = []
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

for intent in bot["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.wordpunct_tokenize(pattern) #Tokenizing the pattern
        #Since token is a list no need of appending we just use the function extend
        token =[]
        for t in tokens:
          t = t.lower() 
          if t in punctuations:
            continue
          token.append(t)
        intent_part_x.append(token)
        intent_part_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

In [6]:
intent_part_x

[['hi', 'there'],
 ['hello'],
 ['hola'],
 ['hey'],
 ['hi'],
 ['hello'],
 ['good', 'morning'],
 ['good', 'evening'],
 ['bye'],
 ['see', 'you', 'later'],
 ['goodbye'],
 ['nice', 'chatting', 'to', 'you', 'bye'],
 ['till', 'next', 'time'],
 ['bye', 'chatbot'],
 ['thanks'],
 ['thank', 'you'],
 ['that', 's', 'helpful'],
 ['awesome', 'thanks'],
 ['thanks', 'for', 'helping', 'me'],
 ['q'],
 ['random'],
 ['how', 'you', 'could', 'help', 'me'],
 ['what', 'you', 'can', 'do'],
 ['what', 'help', 'you', 'provide'],
 ['how', 'you', 'can', 'be', 'helpful'],
 ['what', 'support', 'is', 'offered'],
 ['what', 'services', 'do', 'you', 'provide'],
 ['what', 'can', 'you', 'help', 'me', 'with'],
 ['how', 'to', 'reset', 'my', 'password'],
 ['i', '’', 'm', 'unable', 'to', 'log', 'in'],
 ['my', 'password', 'is', 'lost', 'need', 'to', 'reset'],
 ['open', 'password', 'reset', 'module'],
 ['reset', 'my', 'password'],
 ['forgot', 'password'],
 ['how', 'do', 'i', 'reset', 'my', 'password'],
 ['hi', 'i', 'have', 'probl

In [43]:
#vocab size
maxlen = 10  #number of word used from each review
embedding_size = 50

In [44]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=150)
tokenizer.fit_on_texts(intent_part_x)

In [45]:
X = tokenizer.texts_to_sequences(intent_part_x)
X = pad_sequences(X,padding = 'post', maxlen = maxlen)
y = np.asarray(intent_part_y)

print("Number of Samples:", len(X))
print(X[0])
print(X.shape)
print("="*100)
print("Number of Labels: ", len(y))
print(y[0])
print(y.shape)

Number of Samples: 88
[24 60  0  0  0  0  0  0  0  0]
(88, 10)
Number of Labels:  88
greeting
(88,)


In [46]:
vocab_size = 150
EMBEDDING_FILE = './glove.6B.50d.txt'

def get_coefs(word,*arr): 
  return word, np.asarray(arr, dtype='float32')

embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


In [47]:
import pandas as pd
import numpy as np
import gensim
from sklearn.model_selection import train_test_split
import string
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
strlen = lambda x: len(x)
class TextClassification:
    def __init__(self):
        self.punctuations = list(string.punctuation)
        self.remove_list = self.punctuations + ["`", "'", "``", "''",","]
        self.nonascii = lambda text: ''.join([i if ord(i) < 128 else ' ' for i in text])
        self.w2v_model = embeddings_index

    def cnvt_word2vec(self,lines,fixed_sent_len,vec_len):
        default_value = np.zeros(vec_len)

        key_set = self.w2v_model.keys()
        mat = np.zeros((len(lines), fixed_sent_len, vec_len))

        for i in range(0, len(lines)):

            line = lines[i]
            #print ("(" + str(round(float(i)/len(lines) , 4)*100) + "%) completed")
            for j in range(0, fixed_sent_len):
                # j = 0
                if j < len(line) and line[j] in key_set:
                    mat[i,j,:] = self.w2v_model[line[j]][0:vec_len]
                else:
                    mat[i,j,:] = default_value
            #print ("working with "+str(i) + " of "+ str(len(lines)))
        return mat


In [48]:
tc= TextClassification()

In [49]:
embedding_size

50

In [50]:
mat = tc.cnvt_word2vec(intent_part_x,maxlen,embedding_size)


In [51]:
mat.shape

(88, 10, 50)

In [52]:
labels

['greeting',
 'goodbye',
 'thanks',
 'noanswer',
 'options',
 'Password Reset',
 'E-mail id',
 'Blue Sreen issue',
 'Deleted files',
 'Unsaved Document',
 'Slow Computer',
 'Unexpected Shut Down',
 'Printing issues',
 'Vague',
 'Virus',
 'Mouse',
 'USB',
 'Slow Internet']

In [53]:
 from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_labels = le.fit_transform(y)
le.classes_

array(['Blue Sreen issue', 'Deleted files', 'E-mail id', 'Mouse',
       'Password Reset', 'Printing issues', 'Slow Computer',
       'Slow Internet', 'USB', 'Unexpected Shut Down', 'Unsaved Document',
       'Vague', 'Virus', 'goodbye', 'greeting', 'noanswer', 'options',
       'thanks'], dtype='<U20')

In [54]:
from tensorflow.keras.utils import to_categorical
y_labels = to_categorical(y_labels)

In [61]:
from tensorflow.keras.models import Model
def create_convnet(rows,cols,labels):
    inputs = tf.keras.layers.Input(shape=(rows, cols))

    tower_1 = tf.keras.layers.Conv1D(500, 3, padding='valid', activation='relu')(inputs)
    #tower_1 = MaxPooling1D(3)(tower_1)

    tower_2 = tf.keras.layers.Conv1D(500,4, padding='valid', activation='relu')(inputs)
    #tower_2 = MaxPooling1D(4)(tower_2)

    merged = tf.keras.layers.concatenate([tower_1, tower_2], axis=1)
    merged = tf.keras.layers.Flatten()(merged)
    
    #dense_1 = Dense(100, activation='relu')(merged)
    out = tf.keras.layers.Dense(labels, activation='softmax')(merged)
    

    model = Model(inputs, out)
    #from keras.utils.vis_utils import plot_model
    #plot_model(model, to_file='cnn_model.png', show_shapes=True, show_layer_names=True)
    return model

model =create_convnet(maxlen,embedding_size,len(y_labels[0]))

In [62]:

embedding_size

50

In [63]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [64]:
checkpoint_filepath = "model.pkl"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True)


In [65]:
model.fit(mat, y_labels, epochs=20, batch_size=5, verbose=1,callbacks=[model_checkpoint_callback])


Epoch 1/20
18/18 [==============================] - 0s 10ms/step - loss: 3.0156 - accuracy: 0.1818
Epoch 2/20
18/18 [==============================] - 0s 10ms/step - loss: 1.5289 - accuracy: 0.7159
Epoch 3/20
18/18 [==============================] - 0s 8ms/step - loss: 0.7101 - accuracy: 0.8409
Epoch 4/20
18/18 [==============================] - 0s 8ms/step - loss: 0.4031 - accuracy: 0.8864
Epoch 5/20
18/18 [==============================] - 0s 10ms/step - loss: 0.2299 - accuracy: 0.9432
Epoch 6/20
18/18 [==============================] - 0s 8ms/step - loss: 0.1591 - accuracy: 0.9659
Epoch 7/20
18/18 [==============================] - 0s 9ms/step - loss: 0.1098 - accuracy: 0.9659
Epoch 8/20
18/18 [==============================] - 0s 9ms/step - loss: 0.0888 - accuracy: 0.9886
Epoch 9/20
18/18 [==============================] - 0s 10ms/step - loss: 0.0698 - accuracy: 0.9886
Epoch 10/20
18/18 [==============================] - 0s 5ms/step - loss: 0.0752 - accuracy: 0.9773
Epoch 11/20
18/

In [66]:
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 10, 50)]     0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 8, 500)       75500       input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 7, 500)       100500      input_3[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 15, 500)      0           conv1d_4[0][0]                   
                                                                 conv1d_5[0][0]        

In [67]:
model.load_weights(checkpoint_filepath)

In [68]:
def bag_of_words(sentence):
    sentence_token = nltk.wordpunct_tokenize(sentence)
    sentence_token = [word.lower() for word in sentence_token]
    return np.array(sentence_token)

In [69]:
def chat():
    print("Start Talking with the Bot,To Stop type quit")
    while True:
        inp = input("You :   ")
        if inp.lower() == "quit":
            break 

        input_data = [bag_of_words(inp)]
        input_data = tc.cnvt_word2vec(input_data,maxlen,embedding_size)
        results = model.predict(input_data)[0]
        results_index = np.argmax(results)
        tag = le.classes_[results_index]
       
        if results[results_index] >= 0.05:
            #Looping through the json file
            for tags in bot["intents"]:
                if tags["tag"] == tag:
                    responses = tags["responses"]
            print(np.random.choice(responses))
        
        else:
            print("I dont quite Understand, Ask another question")

In [70]:
def talk(inp):
      input_data = [bag_of_words(inp)]
      input_data = tc.cnvt_word2vec(input_data,maxlen,embedding_size)
      results = model.predict(input_data)[0]
      results_index = np.argmax(results)
      tag = le.classes_[results_index]
      
      if results[results_index] >= 0.05:
          #Looping through the json file
          for tags in bot["intents"]:
              if tags["tag"] == tag:
                  responses = tags["responses"]
          return(np.random.choice(responses))
      
      else:
          return("I dont quite Understand, Ask another question")

In [71]:
chat()

Start Talking with the Bot,To Stop type quit
You :   hi
Hi there, how can I help?
You :   can you dance?
I can guide you through
 1)Password Reset
2) Trouble-Shooting issues
3) Virus Issues
4) Printing Issues
5) many more IT issues
You :   blue screen of eath
rebooting can help the issue
You :   usb issues
First things to check are:
1) Does the device work in a different USB port on the machine?
2) Are other devices recognised in that port?
3) Does the device work on another user’s machine?
 If you have tried these troubleshooting methods and still no luck, then your IT support help-desk can proceed with some more in-depth troubleshooting.
You :   wifi
Not sure I understand
You :   wifi troubles
The reset pin has been sent on your registered mobile number


KeyboardInterrupt: ignored

In [72]:
!pip install flask_ngrok

In [73]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request
from flask import Flask
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():
    return render_template("index.html") 
@app.route("/get")
def get_bot_response():    
    userText = request.args.get('msg')  
    print(userText)
    response = str(talk(userText))
    print(response)  
    return response
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4486811f2e48.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [08/Aug/2020 07:51:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2020 07:51:05] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2020 07:51:06] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Aug/2020 07:51:10] "GET /get?msg=hi HTTP/1.1" 200 -


hi
Hi there, how can I help?


127.0.0.1 - - [08/Aug/2020 07:51:23] "GET /get?msg=can%20you%20dance%3F HTTP/1.1" 200 -


can you dance?
Offering support for 
1) Password Reset
2) Trouble-Shooting issues
3) Virus Issues
4) Printer issues
5) Other IT tasks


127.0.0.1 - - [08/Aug/2020 07:51:35] "GET /get?msg=blue%20screen%20of%20deth HTTP/1.1" 200 -


blue screen of deth
Simply rebooting can fix the blue screen of death (or STOP error, as it is otherwise known).
